## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
import pandas as pd
import numpy as np


# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

'wget' is not recognized as an internal or external command,
operable program or batch file.


1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [2]:

df = pd.read_csv("data_000637.txt", nrows=20000) 
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
19995,1,0,27,3869200366,2513,29
19996,1,0,63,3869200366,2517,6
19997,1,0,32,3869200366,2519,5
19998,1,0,17,3869200366,2522,21


In [3]:

column = df["BX_COUNTER"]
x = column.max()
print(x)

3563


In [4]:
mk=pd.DataFrame(df[df['FPGA']==1])
mk['time_ns']=mk['TDC_MEAS']*25/30+mk['BX_COUNTER']*25
ns=mk['time_ns']
time=ns.sum()-ns.min()
print('The data taking lasted',time*pow(10,-9),'seconds')

The data taking lasted 0.2736819175 seconds


In [5]:

mk

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns
10,1,1,7,3869200167,2785,4,69628.333333
12,1,1,6,3869200167,2792,18,69815.000000
15,1,1,139,3869200167,2797,0,69925.000000
16,1,1,8,3869200167,2787,14,69686.666667
18,1,1,5,3869200167,2795,4,69878.333333
...,...,...,...,...,...,...,...
19933,1,1,1,3869200365,2275,2,56876.666667
19936,1,1,2,3869200365,2278,14,56961.666667
19951,1,1,1,3869200365,2780,17,69514.166667
19954,1,1,20,3869200365,2782,17,69564.166667


In [6]:
#Replace the values (all 1) of the HEAD column randomly with 0 or 1.
df['HEAD']=np.random.randint(2, size=len(df))
df['HEAD']

0        0
1        1
2        1
3        1
4        0
        ..
19995    0
19996    1
19997    0
19998    1
19999    1
Name: HEAD, Length: 20000, dtype: int32

In [7]:
# Create a new DataFrame that contains only the rows with HEAD=1
mask=(df['HEAD']==1)
headOne=pd.DataFrame(df[mask])
headOne

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
5,1,0,63,3869200167,2762,4
9,1,0,60,3869200167,2788,7
...,...,...,...,...,...,...
19992,1,0,40,3869200366,2388,28
19994,1,0,21,3869200366,2513,23
19996,1,0,63,3869200366,2517,6
19998,1,0,17,3869200366,2522,21


In [8]:
# Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel
df['HEAD']=1
fpga1=df[df['FPGA']==1]
fpga2=df[df['FPGA']==0]
dtc1=fpga1['TDC_CHANNEL']
dtc2=fpga2['TDC_CHANNEL']

import matplotlib.pyplot as plt

plt.hist(dtc1, bins=np.arange(dtc1.min(), dtc1.max()), color='#0504aa', alpha=0.7)
plt.show()
plt.hist(dtc2, bins=np.arange(dtc2.min(), dtc2.max()), color='#0504aa', alpha=0.7)
plt.xlabel('Value')
plt.ylabel('Frequency')

<Figure size 640x480 with 1 Axes>

Text(0, 0.5, 'Frequency')

In [9]:
# Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)
C=fpga1.groupby('TDC_CHANNEL').sum()
noisy1=C['FPGA'].nlargest(3)
print(noisy1)
D=fpga2.groupby('TDC_CHANNEL').sum()
noisy2=D['HEAD'].nlargest(3)
print(noisy2)

TDC_CHANNEL
139    522
2      479
1      435
Name: FPGA, dtype: int64
TDC_CHANNEL
139    1171
64      975
63      961
Name: HEAD, dtype: int64


In [10]:
# Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

ok=fpga1.groupby('TDC_CHANNEL')['ORBIT_CNT'].nunique()
print(ok.max())
ko=fpga2.groupby('TDC_CHANNEL')['ORBIT_CNT'].nunique()
ko.max()

187


200